> https://docs.langchain.com/oss/python/langgraph/interrupts

When an interrupt is triggered, LangGraph saves the graph state using its `persistence` layer and waits indefinitely until you resume execution.

# Pause using `interrupt`

To use interrupt, you need:
1. A **checkpointer** to persist the graph state (use a durable checkpointer in production)
2. A **thread ID** in your config so the runtime knows which state to resume from
3. To call `interrupt()` where you want to pause (payload must be JSON-serializable)

In [11]:
from typing import TypedDict
from langgraph.types import interrupt

class State(TypedDict):
    approved: bool


def placeholder_node(state: State):
    print(f"placeholder: {state}")
    return state


def approval_node(state: State):
    # Pause and ask for approval
    approved = interrupt("Do you approve this action?")
    print(approved)

    # When you resume, Command(resume=...) returns that value here
    return {"approved": approved}

When you call interrupt, here’s what happens:
1. **Graph execution gets suspended** at the exact point where `interrupt` is called
2. **State is saved** using the checkpointer so execution can be resumed later, In production, this should be a persistent checkpointer (e.g. backed by a database)
3. **Value is returned** to the caller under `__interrupt__`; it can be any JSON-serializable value (string, object, array, etc.)
4. **Graph waits indefinitely** until you resume execution with a response
5. **Response is passed back** into the node when you resume, becoming the return value of the `interrupt()` call

# Resuming interrupts

After an interrupt pauses execution, you resume the graph by invoking it again with a `Command` that contains the resume value. The resume value is passed back to the `interrupt` call, allowing the node to continue execution with the external input.

In [12]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command

graph = (
    StateGraph(State)
    .add_node(placeholder_node)
    .add_node(approval_node)
    .add_edge(START, "placeholder_node")
    .add_edge("placeholder_node", "approval_node")
    .add_edge("approval_node", END)
    .compile(checkpointer=InMemorySaver())
)

# Initial run - hits the interrupt and pauses
# thread_id is the persistent pointer (stores a stable ID in production)
config = {"configurable": {"thread_id": "thread-1"}}
result = graph.invoke({"input": "data"}, config=config)

# Check what was interrupted
# __interrupt__ contains the payload that was passed to interrupt()
print(result["__interrupt__"])
# > [Interrupt(value='Do you approve this action?')]

# Resume with the human's response
# The resume payload becomes the return value of interrupt() inside the node
graph.invoke(Command(resume=True), config=config)

placeholder: {}
[Interrupt(value='Do you approve this action?', id='d4b0109ec7ac8304d215caa6570fe339')]
True


{'approved': True}

Key points about resuming:
- You must use the **same thread ID** when resuming that was used when the interrupt occurred
- The value passed to `Command(resume=...)` becomes the return value of the `interrupt` call
- The node restarts from the beginning of the node where the `interrupt` was called when resumed, so any code before the `interrupt` runs again
- You can pass any JSON-serializable value as the resume value

# Common patterns

## Approve or reject

In [21]:
from typing import Literal, Optional, TypedDict

from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command, interrupt


class ApprovalState(TypedDict):
    action_details: str
    status: Optional[Literal["pending", "approved", "rejected"]]


def approval_node(state: ApprovalState) -> Command[Literal["proceed", "cancel"]]:
    # Expose details so the caller can render them in a UI
    decision = interrupt({
        "question": "Approve this action?",
        "details": state["action_details"],
    })

    # Route to the appropriate node after resume
    return Command(goto="proceed" if decision else "cancel")


def proceed_node(state: ApprovalState):
    state["status"] = "approved"
    return {"status": f"in proceed_node, state={state}"}


def cancel_node(state: ApprovalState):
    state["status"] = "rejected"
    return {"status": f"in cancel_node, state={state}"}


builder = StateGraph(ApprovalState)
builder.add_node("approval", approval_node)
builder.add_node("proceed", proceed_node)
builder.add_node("cancel", cancel_node)
builder.add_edge(START, "approval")
builder.add_edge("proceed", END)
builder.add_edge("cancel", END)

# Use a more durable checkpointer in production
checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

config = {"configurable": {"thread_id": "approval-123"}}
initial = graph.invoke(
    {"action_details": "Transfer $500", "status": "pending"},
    config=config,
)
print(initial["__interrupt__"])  # -> [Interrupt(value={'question': ..., 'details': ...})]

# Resume with the decision; True routes to proceed, False to cancel
resumed = graph.invoke(Command(resume=True), config=config)
print(resumed["status"])  # -> "approved"

[Interrupt(value={'question': 'Approve this action?', 'details': 'Transfer $500'}, id='7bed1482ac30b0d46898ef7bc1fe934e')]
in proceed_node, state={'action_details': 'Transfer $500', 'status': 'approved'}


## Review and edit state

In [22]:
from typing import TypedDict

from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command, interrupt


class ReviewState(TypedDict):
    generated_text: str


def review_node(state: ReviewState):
    # Ask a reviewer to edit the generated content
    updated = interrupt({
        "instruction": "Review and edit this content",
        "content": state["generated_text"],
    })
    return {"generated_text": updated}


builder = StateGraph(ReviewState)
builder.add_node("review", review_node)
builder.add_edge(START, "review")
builder.add_edge("review", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

config = {"configurable": {"thread_id": "review-42"}}
initial = graph.invoke({"generated_text": "Initial draft"}, config=config)
print(initial["__interrupt__"])  # -> [Interrupt(value={'instruction': ..., 'content': ...})]

# Resume with the edited text from the reviewer
final_state = graph.invoke(
    Command(resume="Improved draft after review"),
    config=config,
)
print(final_state["generated_text"])  # -> "Improved draft after review"

[Interrupt(value={'instruction': 'Review and edit this content', 'content': 'Initial draft'}, id='b65781a10b98c756b14a6667fb4eeb5c')]
Improved draft after review


# Interrupts in tools

In [ ]:
from typing import TypedDict

from langchain.tools import tool
from langchain_deepseek import ChatDeepSeek
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command, interrupt


class AgentState(TypedDict):
    messages: list[dict]


@tool
def send_email(to: str, subject: str, body: str):
    """Send an email to a recipient."""
    # Pause before sending; payload surfaces in result["__interrupt__"]
    response = interrupt({
        "action": "send_email",
        "to": to,
        "subject": subject,
        "body": body,
        "message": "Approve sending this email?",
    })

    if response.get("action") == "approve":
        final_to = response.get("to", to)
        final_subject = response.get("subject", subject)
        final_body = response.get("body", body)

        # Actually send the email (your implementation here)
        print(f"[send_email] to={final_to} subject={final_subject} body={final_body}")
        return f"Email sent to {final_to}"

    return "Email cancelled by user"


model = ChatDeepSeek(model="deepseek-chat").bind_tools([send_email])


def agent_node(state: AgentState):
    # LLM may decide to call the tool; interrupt pauses before sending
    result = model.invoke(state["messages"])
    return {"messages": state["messages"] + [result]}


builder = StateGraph(AgentState)
builder.add_node("agent", agent_node)
builder.add_edge(START, "agent")
builder.add_edge("agent", END)

graph = builder.compile(checkpointer=InMemorySaver())

config = {"configurable": {"thread_id": "email-workflow"}}
initial = graph.invoke(
    {
        "messages": [
            {
                "role": "user", 
                "content": """to: alice@example.com
                subject: about our meeting
                body: Dear Alice, \nI'm sorry to tell you that I just caught a fever yesterday. Could you please reschedule the meeting to next Monday?\nBest Regards,\nJohn
                """
            }
        ]
    },
    config=config,
)
print(initial["__interrupt__"])  # -> [Interrupt(value={'action': 'send_email', ...})]

# Resume with approval and optionally edited arguments
resumed = graph.invoke(
    Command(resume={"action": "approve", "subject": "Updated subject"}),
    config=config,
)
print(resumed["messages"][-1])  # -> Tool result returned by send_email

## Validating human input

In [39]:
from typing import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.types import Command, interrupt


class FormState(TypedDict):
    age: int | None


def get_age_node(state: FormState):
    prompt = "What is your age?"

    while True:
        answer = interrupt(prompt)  # payload surfaces in result["__interrupt__"]

        if isinstance(answer, int) and answer > 0:
            return {"age": answer}

        prompt = f"'{answer}' is not a valid age. Please enter a positive number."


builder = StateGraph(FormState)
builder.add_node("collect_age", get_age_node)
builder.add_edge(START, "collect_age")
builder.add_edge("collect_age", END)

graph = builder.compile(checkpointer=InMemorySaver())

config = {"configurable": {"thread_id": "form-1"}}
first = graph.invoke({"age": None}, config=config)
print(first["__interrupt__"])  # -> [Interrupt(value='What is your age?', ...)]

# Provide invalid data; the node re-prompts
retry = graph.invoke(Command(resume="thirty"), config=config)
print(retry["__interrupt__"])  # -> [Interrupt(value="'thirty' is not a valid age...", ...)]

# Provide valid data; loop exits and state updates
final = graph.invoke(Command(resume=30), config=config)
print(final["age"])  # -> 30

[Interrupt(value='What is your age?', id='a8c352a2d708837b9b22eed076b9b9aa')]
[Interrupt(value="'thirty' is not a valid age. Please enter a positive number.", id='a8c352a2d708837b9b22eed076b9b9aa')]
30


# Rules of interrupts

## Do not wrap `interrupt` calls in try/except

The way that `interrupt` pauses execution at the point of the call is by throwing a special exception. If you wrap the `interrupt` call in a try/except block, you will catch this exception and the interrupt will not be passed back to the graph.

## Do not reorder `interrupt` calls within a node

- ✅ Keep interrupt calls consistent across node executions
- 🔴 Do not conditionally skip interrupt calls within a node
- 🔴 Do not loop interrupt calls using logic that isn’t deterministic across executions

## Do not return complex values in `interrupt` calls

Depending on which checkpointer is used, complex values may not be serializable (e.g. you can’t serialize a function). To make your graphs adaptable to any deployment, it’s best practice to only use values that can be reasonably serialized.

## Side effects called before `interrupt` must be idempotent

Because interrupts work by re-running the nodes they were called from, side effects called before `interrupt` should (ideally) be idempotent. For context, idempotency means that the same operation can be applied multiple times without changing the result beyond the initial execution.

As an example, you might have an API call to update a record inside of a node. If `interrupt` is called after that call is made, it will be re-run multiple times when the node is resumed, potentially overwriting the initial update or creating duplicate records.